<h1>Arquivo de Depuração do Modelo</h1>

In [2]:
# Importa o código principal
import sys
sys.path.append('C:\\Users\\Danilo Santos\\Desktop\\Qualificação PPGCC\\abordagem\\RFNS')

# from grimoire.EnginneringForest import EnginneringForest
from grimoire.ClassifierEnginneringForest import ClassifierEnginneringForest
from grimoire.LoggerEnginnering import LoggerEnginnering

In [1]:
#from pandas import DataFrame, Series
import pandas as pd

import numpy as np

import time

from sklearn.model_selection import train_test_split

In [3]:
df_heart = pd.read_csv('../datasets/heart.csv',
                       engine='c', 
                       memory_map=True, 
                       low_memory=True)

In [15]:
df_heart.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
X=df_heart[['age', 'sex', 'cp', 'trestbps',  'chol', 'fbs', 'restecg', 
            'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]
# Labels
y=df_heart['target']

In [17]:
X.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [43]:
X.shape

(303, 13)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, 
                                                        random_state=100, 
                                                        shuffle=True, 
                                                        stratify=y)

In [47]:
print('X train: ', X_train.shape)
print('X_test: ', X_test.shape)

X train:  (212, 13)
X_test:  (91, 13)


<h2>Código do modelo</h2>

In [12]:
class EnginneringForest(ClassifierEnginneringForest):
    
    __slots__ = ('estimators_', 'select_features_', 'group_features_', 
                 'df_predict_', 'n_features_', 'n_samples_', 'name_features_',
                 'prefix_column_predict', 'logger', 'blockpredict')
    
    def __init__(self, select_features: int, reset_log=False, name_log='enginnering.log'):
        if type(select_features) != int:
            raise TypeError('Expectd value int in select_features')
        
        self.estimators_ = []
        self.select_features_ = select_features
        self.group_features_ = []
        self.df_predict_ = []
        self.n_features_ = 0
        self.n_samples_ = 0
        self.name_features_ = []
        #self.prefix_column_predict = 'cls'
        self.logger = LoggerEnginnering(name='enginnering', 
                                        log_file=name_log,
                                        drop_old=reset_log)
        self.blockpredict = []
        super().__init__()
        
    def __del__(self):
        del self.estimators_
        del self.select_features_
        del self.group_features_
        del self.df_predict_
        del self.n_features_
        del self.n_samples_
        del self.name_features_
        #del self.prefix_column_predict

    def build(self, features_set: list) -> None:
        """ Cria um vetor com o número de árvores igual ao número de 
            subconjuntos possíveis """
        self.group_features_ = self.get_arrangement_features(features_set,
                                                             self.select_features_)
        self.group_features_ = self.get_pack_nparray(self.group_features_)
        n_estimator = len(self.group_features_)
        self.estimators_ = self.make_lote_base_estimator(n_estimator)
        self.estimators_ = self.get_pack_nparray(self.estimators_)
        
        #print('N group features: ', len(self.group_features_))
        #print('N estimators: ', len(self.estimators_))
        #print('Group features: \n{0}'.format(self.group_features_))

    def train(self, group_feature: list, estimator):
        msg = 'Training subset = {0}, Timing = {1}, Size (Kb) = {2}'
        start_train = time.time()
        
        subset_xdata, subset_ydata = self.get_subset(group_feature)
        fit_ = estimator.fit(subset_xdata, subset_ydata)
        
        #print('Train = {0}, Length X = {1}, Length Y = {2}'.format(group_feature, subset_xdata.shape, subset_ydata.shape))
        
        del subset_xdata
        del subset_ydata
        
        end_train = time.time()
        self.logger.add('debug',msg.format(group_feature, 
                                           (end_train - start_train),
                                           self.get_size_estimator(fit_)))
        
        return fit_

    def fit(self, X, y) -> None:
        from pandas import DataFrame, Series
        if not isinstance(X, DataFrame):
            raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
        if not isinstance(y, Series):
            raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
            
        self.n_samples_, self.n_features_ = X.shape
        self.name_features_ = X.columns
        
        self.train_X = X
        self.train_y = y
        
        #print('N samples: ',  self.n_samples_)
        #print('N features: ', self.n_features_)
        #print('Name features: ', self.name_features_)
        
        # Cria a floresta
        self.build(features_set=self.name_features_)

        # Treina as arvores individualmente
        self.estimators_ = [self.train(subset_feature, estimator) 
                            for subset_feature, 
                                estimator in zip(self.group_features_, 
                                                 self.estimators_)]
        self.estimators_ = self.get_pack_nparray(self.estimators_)
        
        del self.train_X
        del self.train_y
    
    def voting(self, data) -> list:
        final_predict = []
        for instance in data:
            marjotiry = (instance.sum() >= (instance.shape[1] / 2)) and 1 or 0
            final_predict.append(marjotiry)
        return final_predict
        
    def predict(self, X) -> list:
        from pandas import DataFrame
        if not isinstance(X, DataFrame):
            raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
        # Este novo código se baseia em lidar com um volume muito grande para predição
        self.predict_X = X.reset_index()
        del X
        self.logger.add('debug','Size predict = {}'.format(self.predict_X.shape))
                                              
        self.logger.add('debug','N estimators = {}'.format(len(self.estimators_)))
        
        for x_, y_ in self.get_block_fit():
            self.logger.add('debug','Block Limit = ({}, {})'.format(x_, y_))
            
            dfsub = self.predict_X.loc[x_:y_]
            block_predict = []
            
            for subset_feature, estimator in zip(self.group_features_, self.estimators_):
                self.logger.add('debug', 'Subset predict = {0}'.format(subset_feature))
                subset_test = dfsub.loc[:, subset_feature]
                block_predict.append(estimator.predict(subset_test))
            
            block_predict = np.matrix(block_predict)
            self.logger.add('debug', "Shape One = {0}".format(block_predict.shape))
            
            block_predict = block_predict.T
            self.logger.add('debug', "Shape Two = {0}".format(block_predict.shape))
            self.logger.add('debug', "Block predict \n{0}".format(block_predict))
            
            # Guarda a matrix de predição
            #np.savetxt('dump_sf_{0}_bp_{1}'.format(self.select_features_, (x_,y_)), 
            #           block_predict, delimiter=",")
            
            self.blockpredict = pd.DataFrame(block_predict)
            
            # chama o voting na matriz de predições
            block_voting = self.voting(block_predict)
            self.logger.add('debug', "Block voting data \n{0}".format(str(block_voting)))
            self.logger.add('debug', "Block voting len {0}".format(len(block_voting)))
            self.df_predict_.extend(block_voting)
        
        return self.df_predict_

In [13]:
model_eg = EnginneringForest(3)

In [14]:
model_eg.fit(X_train, y_train)

In [15]:
y_pred = model_eg.predict(X_test)

In [16]:
model_eg.blockpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,276,277,278,279,280,281,282,283,284,285
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
2,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,0,1,1,0,...,0,0,0,0,1,0,0,1,0,0


In [26]:
model_eg.blockpredict.shape

(91, 288)

In [17]:
block = model_eg.blockpredict

In [18]:
block.insert(loc=286, column='predict', value=y_pred)

In [19]:
block.insert(loc=287, column='real', value=y_test)

In [23]:
my_y = y_test
my_y.reset_index
block['real'] = my_y

In [24]:
block.head()

,0,1,2,3,4,5,6,7,8,9,...,278,279,280,281,282,283,284,285,predict,real
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1.0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,NaN
2,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,1,1,1,1,1,1,0,1,1,0,...,0,0,1,0,0,1,0,0,1,NaN


In [25]:
print(block.loc[0])

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
5          0.0
6          0.0
7          0.0
8          0.0
9          0.0
10         1.0
11         0.0
12         0.0
13         0.0
14         0.0
15         0.0
16         0.0
17         0.0
18         0.0
19         0.0
20         0.0
21         1.0
22         0.0
23         0.0
24         0.0
25         0.0
26         1.0
27         0.0
28         0.0
29         1.0
          ... 
258        0.0
259        0.0
260        0.0
261        0.0
262        1.0
263        0.0
264        1.0
265        0.0
266        0.0
267        1.0
268        0.0
269        1.0
270        0.0
271        0.0
272        1.0
273        0.0
274        1.0
275        0.0
276        0.0
277        0.0
278        1.0
279        0.0
280        0.0
281        0.0
282        0.0
283        1.0
284        0.0
285        0.0
predict    0.0
real       1.0
Name: 0, Length: 288, dtype: float64


In [49]:
# testa o train
model_eg = EnginneringForest(3)

In [50]:
model_eg.fit(X_train, y_train)

Train = ['age' 'sex' 'cp'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'sex' 'trestbps'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'sex' 'chol'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'sex' 'fbs'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'sex' 'restecg'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'sex' 'thalach'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'sex' 'exang'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'sex' 'oldpeak'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'sex' 'slope'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'sex' 'ca'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'sex' 'thal'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'cp' 'trestbps'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'cp' 'chol'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'cp' 'fbs'], Length X = (212, 3), Length Y = (212,)
Train = ['age' 'cp' 'res

Train = ['cp' 'trestbps' 'thalach'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'trestbps' 'exang'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'trestbps' 'oldpeak'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'trestbps' 'slope'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'trestbps' 'ca'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'trestbps' 'thal'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'chol' 'fbs'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'chol' 'restecg'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'chol' 'thalach'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'chol' 'exang'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'chol' 'oldpeak'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'chol' 'slope'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'chol' 'ca'], Length X = (212, 3), Length Y = (212,)
Train = ['cp' 'chol' 'thal'], Length X = (212, 3), Length Y = (21

Train = ['fbs' 'oldpeak' 'slope'], Length X = (212, 3), Length Y = (212,)
Train = ['fbs' 'oldpeak' 'ca'], Length X = (212, 3), Length Y = (212,)
Train = ['fbs' 'oldpeak' 'thal'], Length X = (212, 3), Length Y = (212,)
Train = ['fbs' 'slope' 'ca'], Length X = (212, 3), Length Y = (212,)
Train = ['fbs' 'slope' 'thal'], Length X = (212, 3), Length Y = (212,)
Train = ['fbs' 'ca' 'thal'], Length X = (212, 3), Length Y = (212,)
Train = ['restecg' 'thalach' 'exang'], Length X = (212, 3), Length Y = (212,)
Train = ['restecg' 'thalach' 'oldpeak'], Length X = (212, 3), Length Y = (212,)
Train = ['restecg' 'thalach' 'slope'], Length X = (212, 3), Length Y = (212,)
Train = ['restecg' 'thalach' 'ca'], Length X = (212, 3), Length Y = (212,)
Train = ['restecg' 'thalach' 'thal'], Length X = (212, 3), Length Y = (212,)
Train = ['restecg' 'exang' 'oldpeak'], Length X = (212, 3), Length Y = (212,)
Train = ['restecg' 'exang' 'slope'], Length X = (212, 3), Length Y = (212,)
Train = ['restecg' 'exang' 'ca']

In [45]:
# testa o fit
model_eg = EnginneringForest(3)

In [46]:
model_eg.fit(X_train, y_train)

N samples:  212
N features:  13
Name features:  Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal'],
      dtype='object')


In [40]:
# testa o build
model_eg = EnginneringForest(3)

In [41]:
model_eg.fit(X_train, y_train)

N group features:  286
N estimators:  286
Group features: 
[['age' 'sex' 'cp']
 ['age' 'sex' 'trestbps']
 ['age' 'sex' 'chol']
 ['age' 'sex' 'fbs']
 ['age' 'sex' 'restecg']
 ['age' 'sex' 'thalach']
 ['age' 'sex' 'exang']
 ['age' 'sex' 'oldpeak']
 ['age' 'sex' 'slope']
 ['age' 'sex' 'ca']
 ['age' 'sex' 'thal']
 ['age' 'cp' 'trestbps']
 ['age' 'cp' 'chol']
 ['age' 'cp' 'fbs']
 ['age' 'cp' 'restecg']
 ['age' 'cp' 'thalach']
 ['age' 'cp' 'exang']
 ['age' 'cp' 'oldpeak']
 ['age' 'cp' 'slope']
 ['age' 'cp' 'ca']
 ['age' 'cp' 'thal']
 ['age' 'trestbps' 'chol']
 ['age' 'trestbps' 'fbs']
 ['age' 'trestbps' 'restecg']
 ['age' 'trestbps' 'thalach']
 ['age' 'trestbps' 'exang']
 ['age' 'trestbps' 'oldpeak']
 ['age' 'trestbps' 'slope']
 ['age' 'trestbps' 'ca']
 ['age' 'trestbps' 'thal']
 ['age' 'chol' 'fbs']
 ['age' 'chol' 'restecg']
 ['age' 'chol' 'thalach']
 ['age' 'chol' 'exang']
 ['age' 'chol' 'oldpeak']
 ['age' 'chol' 'slope']
 ['age' 'chol' 'ca']
 ['age' 'chol' 'thal']
 ['age' 'fbs' 'restecg']

In [20]:
print('N estimators: ', len(model_eg.estimators_))

N estimators:  0


In [22]:
print('N select features: ', model_eg.select_features_)

N select features:  3


In [23]:
print('N group features: ', len(model_eg.group_features_))

N group features:  0


In [24]:
print('Predict: ', len(model_eg.df_predict_))

Predict:  0


In [25]:
print('N features: ', model_eg.n_features_)

N features:  0


In [26]:
print('N samples: ', model_eg.n_samples_)

N samples:  0


In [27]:
print('Name features: ', len(model_eg.name_features_), model_eg.name_features_)

Name features:  0 []


In [32]:
model_eg.fit(X_train, y_train)

In [33]:
y_pred = model_eg.predict(X_test)

In [36]:
i = 0
for x, y in zip(y_test, y_pred):
    print("Instance = {0}\t\tText = {1}\t-\tPred = {2}".format(i, x, y))
    i = i + 1

Instance = 0		Text = 0	-	Pred = 0
Instance = 1		Text = 1	-	Pred = 1
Instance = 2		Text = 1	-	Pred = 1
Instance = 3		Text = 0	-	Pred = 0
Instance = 4		Text = 1	-	Pred = 1
Instance = 5		Text = 1	-	Pred = 1
Instance = 6		Text = 0	-	Pred = 1
Instance = 7		Text = 1	-	Pred = 1
Instance = 8		Text = 1	-	Pred = 1
Instance = 9		Text = 0	-	Pred = 0
Instance = 10		Text = 1	-	Pred = 1
Instance = 11		Text = 0	-	Pred = 1
Instance = 12		Text = 1	-	Pred = 1
Instance = 13		Text = 1	-	Pred = 1
Instance = 14		Text = 1	-	Pred = 1
Instance = 15		Text = 0	-	Pred = 1
Instance = 16		Text = 1	-	Pred = 1
Instance = 17		Text = 1	-	Pred = 1
Instance = 18		Text = 1	-	Pred = 1
Instance = 19		Text = 0	-	Pred = 0
Instance = 20		Text = 1	-	Pred = 1
Instance = 21		Text = 0	-	Pred = 0
Instance = 22		Text = 0	-	Pred = 1
Instance = 23		Text = 1	-	Pred = 0
Instance = 24		Text = 0	-	Pred = 0
Instance = 25		Text = 1	-	Pred = 1
Instance = 26		Text = 1	-	Pred = 1
Instance = 27		Text = 0	-	Pred = 1
Instance = 28		Text = 0	-	Pred